In [1]:
import os
import pandas as pd
import numpy as np


mode = 'no_class'
distance_functions = ['euclidian', 'city-block', 'chebyshev']
input_path = os.path.join(
    '../data/generated', 'avila-combined-{}.csv'.format(mode))


data = pd.read_csv(input_path)
data_array = data.to_numpy()






In [2]:
data.head()

,0,1,2,3,4,5,6,7,8,9
0,0.266074,-0.165620,0.320980,0.483299,0.172340,0.273364,0.371178,0.929823,0.251173,0.159345
1,0.130292,0.870736,-3.210528,0.062493,0.261718,1.436060,1.465940,0.636203,0.282354,0.515587
2,-0.116585,0.069915,0.068476,-0.783147,0.261718,0.439463,-0.081827,-0.888236,-0.123005,0.582939
3,0.031541,0.297600,-3.210528,-0.583590,-0.721442,-0.307984,0.710932,1.051693,0.594169,-0.533994
4,0.229043,0.807926,-0.052442,0.082634,0.261718,0.148790,0.635431,0.051062,0.032902,-0.086652


In [3]:
data.shape

(20867, 10)

In [4]:
from scipy.spatial.distance import pdist, squareform

In [5]:
metrics = ['euclidean','cityblock','chebyshev']

In [6]:
def normalize_data(D):
    l = ([np.sum(p[p.sum(axis=0).argmin()] ) for p in list(D)])
    k = np.asarray(l)[...,np.newaxis,np.newaxis]
    X = np.divide(D,k)
    return X

In [7]:
store = pd.HDFStore("diss_matrices_.h5")

for metric in metrics:
    distance_matrix = pdist(data_array, metric=metric)
    df1 = squareform(distance_matrix)
    df1 = df1[np.newaxis,...]
    df1 = normalize_data(df1)[0]
    print(df1.shape)
    print(df1.diagonal())
    df1 = pd.DataFrame(df1)

    store[metric] = df1
    
store.close()
    

(20867, 20867)
[0. 0. 0. ... 0. 0. 0.]
(20867, 20867)
[0. 0. 0. ... 0. 0. 0.]
(20867, 20867)
[0. 0. 0. ... 0. 0. 0.]


In [8]:
d = np.random.rand(3,5,5)
#print(d)
d = normalize_data(d)
print(d.shape)
print(np.sum(d[0],axis=1))


(3, 5, 5)
[1.40725052 1.01241647 1.         0.59849261 0.4617171 ]


In [9]:
Dk = np.arange(30).reshape(3,2,5)
A = np.arange(6).reshape(3,2)
print(A.shape,Dk.shape)
print("A =========================== A ==============")
print(A)
print("Dk =========================== DK ==============")

print(Dk)
print("RESULT =========================== A ==============")

term_1  = np.einsum('pk,pkn->kn',A,Dk)
print(term_1.shape)
print(term_1)
std_term = np.asarray([column/np.sum(column) for column in term_1.T]).T
print(std_term)

(3, 2) (3, 2, 5)
A =========================== A ==============
[[0 1]
 [2 3]
 [4 5]]
Dk =========================== DK ==============
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]]

 [[10 11 12 13 14]
  [15 16 17 18 19]]

 [[20 21 22 23 24]
  [25 26 27 28 29]]]
RESULT =========================== A ==============
(2, 5)
[[100 106 112 118 124]
 [175 184 193 202 211]]
[[0.36363636 0.36551724 0.36721311 0.36875    0.37014925]
 [0.63636364 0.63448276 0.63278689 0.63125    0.62985075]]


In [10]:
output_path = os.path.join(
    '../data/diss_matrices_{}/diss_matrix_{}.csv'.format(mode,metric))


